In [54]:
import pandas as pd
import MetaTrader5 as mt5 
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [34]:
dataset = pd.read_csv("imf-monthly-survey-current.csv")
dataset = dataset.dropna()

In [142]:
dataset['IMF MONTHLY DATA - RESERVE BANK OF AUSTRALIA']

9      I. Official reserve assets and other foreign c...
11                            A. Official reserve assets
12     (1) Foreign currency reserves (in convertible ...
13                                        (a) Securities
15                 (b) total currency and deposits with:
16         (i) other national central banks, BIS and IMF
19     (iii) banks headquartered outside the reportin...
21                              (2) IMF reserve position
22                                              (3) SDRs
23     (4) gold (including gold deposits and, if appr...
24                volume in millions of fine troy ounces
25                    (5) other reserve assets (specify)
26                                 financial derivatives
28                                                 other
29            B. Other foreign currency assets (specify)
31      deposits not included in official reserve assets
33     financial derivatives not included in official...
68                             

In [127]:
official_assets = dataset.T[11]
official_assets = pd.DataFrame(official_assets).reset_index()
official_assets = official_assets.iloc[1:]

In [128]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

In [129]:
AU = get_rates('AUDUSD', mt5.TIMEFRAME_MN1, 107).reset_index(drop = True)

In [130]:
AU['close']

0      0.76150
1      0.79138
2      0.76417
3      0.77150
4      0.73086
        ...   
102    0.64346
103    0.63365
104    0.66050
105    0.68104
106    0.67142
Name: close, Length: 107, dtype: float64

In [161]:
combined = pd.concat([official_assets, AU['close']], axis = 1)

In [162]:
combined = combined.drop(columns = ['index'])

In [163]:
for i in range(5):
    combined[f'close_{i}'] = combined['close'].shift(i)

In [164]:
combined = combined.iloc[0:-2]
combined = combined.rename(columns = {11:'Offical reserve'})
combined.corr()

,Offical reserve,close,close_0,close_1,close_2,close_3,close_4
Offical reserve,1.000000,-0.550485,-0.550485,-0.524159,-0.505047,-0.526009,-0.414649
close,-0.550485,1.000000,1.000000,0.877381,0.777947,0.694234,0.671470
close_0,-0.550485,1.000000,1.000000,0.877381,0.777947,0.694234,0.671470
close_1,-0.524159,0.877381,0.877381,1.000000,0.876316,0.774777,0.690398
close_2,-0.505047,0.777947,0.777947,0.876316,1.000000,0.875831,0.775970
close_3,-0.526009,0.694234,0.694234,0.774777,0.875831,1.000000,0.874136
close_4,-0.414649,0.671470,0.671470,0.690398,0.775970,0.874136,1.000000


In [166]:
usdjpy = get_rates('USDJPY', mt5.TIMEFRAME_M5, 20000)
usdjpy

,open,high,low,close
time,,,,
2023-09-28 09:30:00,149.402,149.434,149.389,149.425
2023-09-28 09:35:00,149.424,149.445,149.407,149.408
2023-09-28 09:40:00,149.408,149.429,149.400,149.421
2023-09-28 09:45:00,149.422,149.432,149.407,149.420
2023-09-28 09:50:00,149.422,149.430,149.372,149.373
...,...,...,...,...
2024-01-05 23:25:00,144.714,144.730,144.704,144.719
2024-01-05 23:30:00,144.719,144.750,144.692,144.697
2024-01-05 23:35:00,144.696,144.696,144.669,144.673
